In [1]:
from visualizer import calibration
#TODO: import from waveorder
import visual 

from PyQt5 import QtCore

from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os
import tifffile

from skimage.registration import phase_cross_correlation
import cv2 as cv
from skimage import io
import zarr
import cupy as cp 
import imagej

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
# Load main folder
main_folder =r'F:\Test_ignore\img' #5 L1 moving! amazing neurons.
# Load offsets from desired folder 
processed_files = main_folder + '/'+'processed_files'

#Scope Parameters
# FOV = 50e-6
cam_px = 6.0e-6
totalmag = 15.75
px_size_img = cam_px/totalmag
zstep = 2e-6
z_scale = zstep/px_size_img

### ZARR IMPORT

In [3]:
file_name = 'aligned_stack.zarr'
filepath_save = os.path.join(processed_files,file_name)
zarr_stack = zarr.load(filepath_save)
da_stack = da.from_array(zarr_stack)

In [4]:
viewer= napari.Viewer()
viewer.add_image(da_stack, scale= [z_scale,1,1])

<Image layer 'da_stack' at 0x1ab53dc63a0>

### CROP PSF TO ANALYZE


In [7]:
#Go to napari and make a square for alignment by making a 'Shapes' Layer
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)

[[734.          12.         165.05309394  65.14820383]
 [734.          12.         165.05309394 477.95328513]
 [734.          12.         482.51744393 477.95328513]
 [734.          12.         482.51744393  65.14820383]]
[[165  65]
 [483 478]]
(2, 2)


In [8]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
# np_mip_8bit =visual.im_bit_convert(np_mip,bit=8)
min_val, max_val = template_box
t,c = template[0,0:2]
print(t,c)
crop_img = da_stack[:,:, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
# viewer.add_image(crop_img,name='crop')

734 12


<Image layer 'crop' at 0x2736208f3d0>

### Image J Part

In [3]:
viewer= napari.Viewer()

In [10]:
def start_imagej():
    import imagej
    global ij
    # ij = imagej.init( mode='interactive')
    path = r'C:\Users\yoshi\Documents\Fiji.app'
    ij = imagej.init(path,mode='interactive')
    ij.ui().showUI()
    print(ij.getVersion())

QtCore.QTimer.singleShot(0, start_imagej)

[java.lang.Enum.toString] [WARNING] 1 exceptions occurred during plugin discovery.


2.0.0-rc-69/Inactive


In [11]:
image = np.array(da_stack)

In [12]:
# Align the files
# Retreive the files from Java to Python

ij_stack2  =ij.py.to_java(image)
ij.ui().show('PSF', ij_stack2)

TypeError: No matching overloads found for *static* net.imglib2.img.ImgView.wrap(net.imglib2.python.ReferenceGuardingRandomAccessibleInterval), options are:
	public static net.imglib2.img.Img net.imglib2.img.ImgView.wrap(net.imglib2.RandomAccessibleInterval,net.imglib2.img.ImgFactory)



In [16]:
ij.WindowManager.getActiveWindow()
a =ij.WindowManager.getCurrentImage()
anp = ij.py.from_java(a)
print(anp.shape)
# anp2 = anp[:,:,:,np.newaxis]
# print(anp2.shape)
# anp2 = np.swapaxes(anp,1,4)
# anp2.shape

(60, 342, 413, 25)


In [17]:
anp2 = np.swapaxes(anp,1,3)
print(anp2.shape)
anp2_da = da.asarray(anp2)
viewer.add_image(anp2_da, name='crop',scale=[z_scale,1,1],multiscale=False)

(60, 25, 413, 342)


<Image layer 'crop' at 0x22682c33220>